In [101]:
import pandas as pd
import numpy as np
import random
import yfinance as yf

 When reading the CSV file, the stock value is assumed to be zero until the IPO is launched.

In [2]:
data = pd.read_csv('data.csv')
data.replace(np.nan, 0, inplace = True)

For the sample output, we have selected the stocks of Reliance Industries, SBI, Tata Motors, and HDFC Bank, with the initial investment date set to March 15, 2003, and an initial investment amount of ₹1000.

In [103]:
data_df = data[['Date','RELIANCE','SBIN', 'TATAMOTORS', 'HDFCBANK']]
data_df.set_index(['Date'], inplace = True) 
Investment_date = '2001-03-15'
initial_investment = 1000
data_df = data_df.loc[Investment_date:]
n = data_df.shape[1]

A random allocation of weight distribution among the stocks has been generated for the sample case.

In [37]:
def generate_portfolio_weights(n):
    weights = []
    for i in range(n):
        weights.append(random.random())
    weights = weights/np.sum(weights)
    return weights
weights = generate_portfolio_weights(n)
print(weights)

[0.1602159  0.32719986 0.25197711 0.26060714]


Defining a function that scales the stock value from the initial investment day, adjusting it daily.

In [102]:
def price_scaling(raw_prices_df):
    scaled_prices_df = raw_prices_df.copy()
    for i in raw_prices_df.columns[0:]:
          scaled_prices_df[i] = raw_prices_df[i]/raw_prices_df[i].iloc[0]
    return scaled_prices_df


Scaled up the portfolio of the user.

In [39]:
portfolio_df = data_df.copy()
scaled_df = price_scaling(portfolio_df)
scaled_df

,RELIANCE,SBIN,TATAMOTORS,HDFCBANK
Date,,,,
2001-03-15,1.000000,1.000000,1.000000,1.000000
2001-03-16,0.966646,0.987312,0.960552,0.959356
2001-03-19,0.962870,0.984986,0.984221,0.969053
2001-03-20,0.939836,0.978220,0.961867,0.946771
2001-03-21,1.010195,1.003595,0.972386,0.974417
...,...,...,...,...
2024-07-01,55.000571,37.739445,68.758327,70.361044
2024-07-02,55.177720,37.031229,67.334511,71.409116
2024-07-03,54.728239,37.649797,66.946824,72.979162


Defining a function that tracks the daily value of each stock using the scaled-up dataframe. Additionally, to monitor the portfolio, a separate column is created for the Total Portfolio Value and the daily Percent Change of the portfolio.

In [40]:
def asset_allocation(df, weights, initial_investment):
    for i, stock in enumerate(scaled_df.columns[0:]):
        portfolio_df[stock] = weights[i] * scaled_df[stock]  * initial_investment
    portfolio_df['Portfolio Value'] = portfolio_df.sum(axis =1,   )
    portfolio_df['Portfolio Daily Return [%]'] = portfolio_df['Portfolio Value'].pct_change(1) * 100 
    portfolio_df.replace(np.nan, 0, inplace = True)    
    return portfolio_df

Generating a sample output for the user's portfolio and replacing any misinterpreted values, such as infinity or unavailable data, with 0.

In [82]:
portfolio1_df = asset_allocation(data_df, weights, initial_investment)
portfolio1_df.round(2)
portfolio1_df.replace([np.inf, -np.inf], np.nan, inplace=True)
portfolio1_df.describe()

,RELIANCE,SBIN,TATAMOTORS,HDFCBANK,Portfolio Value,Portfolio Daily Return [%]
count,5809.000000,5809.000000,5809.000000,5809.000000,5.809000e+03,5809.000000
mean,3574.839452,2208.333602,3649.583021,6568.450633,7.811211e+07,-3.312764
std,3720.525851,1604.988556,2853.284623,6606.411383,6.529565e+07,131.009098
min,0.000000,0.000000,0.000000,0.000000,-5.065130e+05,-5690.968192
25%,1273.453922,913.978997,1400.905946,1080.512526,2.613476e+07,-0.795421
50%,2149.453751,2082.941766,2669.706351,3653.874680,5.973811e+07,0.073047
75%,4752.667605,2770.295474,5696.056021,12032.049223,1.113631e+08,1.020568
max,15345.872171,8795.506271,15298.375639,21624.311228,2.905344e+08,108.916436


Defining a function that calculates the return on investment (ROI) based on the initial investment.

In [42]:
def return_on_investment_(df):
    profit = ((df['Portfolio Value'].iloc[-1] - df['Portfolio Value'].iloc[0]) / df['Portfolio Value'].iloc[0]) * 100
    return profit
a = return_on_investment_(portfolio1_df)
print('The return on Investment is', a.round(2), '%')

The return on Investment is 5548.51 %


Defining a function that calculates the historical market return of the market index, in this case, Nifty 50, using the yfinance library.

In [43]:
def historical_market_return(index_symbol, start_date, end_date):
    index_data = yf.download(index_symbol, start=start_date, end=end_date)
    index_data['Daily Return'] = index_data['Adj Close'].pct_change()
    annual_return = index_data['Daily Return'].mean() * 252  
    return annual_return
index_symbol = '^NSEI' 
start_date = Investment_date
end_date = '2024-07-10'
market_return_value = historical_market_return(index_symbol, start_date, end_date)
print(f"Expected Market Return for Nifty 50: {market_return_value:.2%}")

[*********************100%***********************]  1 of 1 completed

Expected Market Return for Nifty 50: 12.64%


Defining a function to calculate the beta of a stock using the yfinance library.

In [44]:
def calculate_beta(stock_ticker, market_ticker, start_date, end_date):
    stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
    market_data = yf.download(market_ticker, start=start_date, end=end_date)
    stock_returns = stock_data['Adj Close'].pct_change().dropna()
    market_returns = market_data['Adj Close'].pct_change().dropna()
    combined_data = pd.concat([stock_returns, market_returns], axis=1)
    combined_data.columns = ['Stock', 'Market']
    combined_data.dropna(inplace=True)
    cov_matrix = np.cov(combined_data['Stock'], combined_data['Market'])
    beta = cov_matrix[0, 1] / cov_matrix[1, 1]
    return beta

Defining a new List for calculating Beta of the stocks.

In [104]:
beta = np.zeros(n)
beta[0] = calculate_beta('RELIANCE.NS', '^NSEI', Investment_date, end_date) 
beta[1] = calculate_beta('SBIN.NS', '^NSEI', Investment_date, end_date)  
beta[2] = calculate_beta('TATAMOTORS.NS', '^NSEI', Investment_date, end_date) 
beta[3] = calculate_beta('HDFCBANK.NS', '^NSEI', Investment_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Defining a new function to calculate expected annual return of the portfolio.

In [105]:
def calculate_annual_return(market_free_return, beta, expected_return):
    return (expected_return + beta * (expected_return - market_free_return))
market_free_return = 0.07
expected_annual_return = np.zeros(4)
for i in range(0,n):
    expected_annual_return[i] = calculate_annual_return(market_free_return, beta[i], market_return_value)

Defining a new function to calculate expected annual portfolio return.

In [106]:
def expected_annual_portfolio_return(expected_annual_return, weights):
    annual_return = 0
    for i in range(0,n):
        annual_return = weights[i] * expected_annual_return[i] + annual_return 
    return annual_return*100
ex = expected_annual_portfolio_return(expected_annual_return, weights)
print(f'The annual expected portfolio return : {ex.round(2)} %')

The annual expected portfolio return : 19.22 %


Defining a new function of a expected volatility. 

In [73]:
def expected_volatility(daily_returns):
    daily_returns.dropna()
    daily_volatility = daily_returns.std()
    annualized_volatility = daily_volatility / np.sqrt(252)
    return annualized_volatility 
cd = expected_volatility(portfolio1_df['Portfolio Daily Return [%]']).round(2) 
print(f'The Expected Volatility of the Portfolio is : {cd} %')

The Expected Volatility of the Portfolio is : 8.27 %


Defining a new function to calculate Sharpe Ratio.

In [74]:
def calculate_sharpe_ratio(rate, market_free_rate, volatility ):
    sharpe_ratio = (rate - market_free_rate * 100 )/(volatility)
    return sharpe_ratio.round(3)
ef = calculate_sharpe_ratio(ex, market_free_return, cd)
print(f'The Sharpe Ratio of the Portfolio is: {ef}')

The Sharpe Ratio of the Portfolio is: 1.465


Creating the Monte Carlo simulation in this cell starting with creating new set of Lists for Weights, Sharpe Ratio, Expected Portfolio Returns, Volatility, Return on Investment and Final Value of the Portfolio. Later defining a new function called Simulation Engine which basically takes the input of weights and intial investment. The function aims to generate every possible result and store it.  

In [84]:
no_of_runs = 10000
weights_runs = np.zeros((no_of_runs, n))
sharpe_ratio_runs = np.zeros(no_of_runs)
expected_portfolio_returns_runs = np.zeros(no_of_runs)
volatility_runs = np.zeros(no_of_runs)
return_on_investment_runs = np.zeros(no_of_runs)
final_value_runs = np.zeros(no_of_runs)
def simulation_engine(weights, initial_investment):
    portfolio3_df = pd.DataFrame()
    portfolio3_df = asset_allocation(data_df, weights, initial_investment)
    portfolio3_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return_on_investment_runs[i] = (portfolio3_df['Portfolio Value'].iloc[-1] - 
                                portfolio3_df['Portfolio Value'].iloc[0]) / portfolio3_df['Portfolio Value'].iloc[0] * 100
    expected_portfolio_returns_runs[i] = expected_annual_portfolio_return(expected_annual_return,weights) 
    volatility_runs[i]= expected_volatility(portfolio3_df['Portfolio Daily Return [%]'])
    sharpe_ratio_runs[i] = calculate_sharpe_ratio(expected_portfolio_returns_runs[i] , market_free_return, volatility_runs[i])
    return return_on_investment_runs[i].round(2), expected_portfolio_returns_runs[i].round(2), volatility_runs[i].round(2), sharpe_ratio_runs[i]

Generating the results

In [85]:
for i in range (no_of_runs):
    weights = generate_portfolio_weights(n)
    weights_runs[i,:] = weights
    return_on_investment_runs[i], expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i] = simulation_engine(weights, initial_investment)

Extracting the iteration with the highest sharpe ratio and similarly the other coefficients. 

In [98]:
m = sharpe_ratio_runs.argmax()
optimal_portfolio_return = return_on_investment_runs[m] * initial_investment
optimal_expected_portfolio_return = expected_portfolio_returns_runs[m] 
optimal_volatility = volatility_runs[m]
optimal_sharpe_ratio = sharpe_ratio_runs[m]
optimal_weights = weights_runs[m]

Final result of the portfolio to understand about the most optimal result.

In [100]:
print(f'The Optimal ratio in which the user must allocate the investment must be: {optimal_weights}')
print(f'This would result in return total: ₹ {optimal_portfolio_return} from the initial investment of ₹{initial_investment}')
print(f'Resulting in net return on investment at: {return_on_investment_runs[m]} %')
print(f'At the annual expected return of : {optimal_expected_portfolio_return} %')
print(f'The expected volatility rate is : {optimal_volatility}')
print(f'The sharpe ratio being: {optimal_sharpe_ratio}')

The Optimal ratio in which the user must allocate the investment must be: [0.00987667 0.07155335 0.89933558 0.0192344 ]
This would result in return total: ₹ 5670080.0 from the initial investment of ₹1000
Resulting in net return on investment at: 5670.08 %
At the annual expected return of : 19.69 %
The expected volatility rate is : 8.25
The sharpe ratio being: 1.537
